# Function Processing Module

## Initialization

### Imports

In [23]:
from typing import Dict, Tuple, Any, Callable

import inspect
import functools

from inspect import isgeneratorfunction

from collections import Counter


from pprint import pprint

import unittest
from pathlib import Path
from typing import List
import re
from sections import Rule, RuleSet
from sections import TriggerEvent
import text_reader as tp

### Type Definitions

In [24]:
Context = Dict[str, Any]


### Exceptions

In [25]:
class FunctionProcessingException(Exception):
    '''Base Exception class for FunctionProcessing.'''

class InvalidArgumentsError(FunctionProcessingException, TypeError):
    '''Raised when a signature wrapped function is called with inappropriate
    arguments.'''


### Utility Functions

In [26]:
def print_sig_info(sig):
    for param in sig.parameters.values():
        if param.default is param.empty:
            dft = ''
        else:
            dft = param.default
        print(f'{param.name}\t{param.kind.description:25s}{dft}')

In [27]:
def print_param_list(sig):
    param_format = '{name:16s}{kind:25s}{default:12s}{annotation:25s}'
    header_dict = {'name': 'Name',
                   'kind': 'Kind',
                   'default': 'Default',
                   'annotation': 'Annotation'}
    print(param_format.format(**header_dict))

    for param in sig.parameters.values():
        if param.default is param.empty:
            dft = ''
        else:
            dft = param.default
        param_attrs = {
            'name': param.name,
            'kind': str(param.kind),
            'default': str(dft),
            'annotation': str(param.annotation)
            }
        print(param_format.format(**param_attrs))

## MapSig Class 
- An instance is a decorator used to wrap functions
- When creating an instance, a "Calling" signature is defined and the resulting wrapped function will appear to have that signature.
- The role of the decorator is to map the parameters in the signature of the wrapped function to the parameters signature for the original un-wrapped function.
- If parameters are present in the wrapped function signature that cannot be mapped to a parameter in the original un-wrapped function signature, those parameter values will be silently dropped.
- If required (positional) parameters are present in the original un-wrapped function signature that cannot be associated with a parameter (of any kind) in the wrapped function signature, an error will be raised when attempting to initialize the object.
- The decorator also assigns appropriate annotation to the wrapped function.
  - The wrapped function annotation specifies the new signature
  - The wrapped function annotation uses the name and doc string from the original un-wrapped function.



There are two reasons for creating this class:
1. To allow built-in or re-purposed functions with a smaller number of parameters to silently drop unnecessary arguments.
2. To provide flexible use of the context argument, as described below.


#### The `context` parameter.
The `context` dictionary is intended to be a flexible way to pass information into and through the sectioning process.  
1. _Processing_ functions can access the context content and modify their behavior accordingly. 
2. In addition, _Processing_ functions in one section can add or modify a context item to direct the behavior of a _Processing_ function in another section.
3. Finally, a _Processing_ functions with a `**kwargs` parameter can match `context` item to a the function's parameters.

In order to accomplish this, the `context` argument must be passed either as a single  object or unpacked using the `**kwargs` notation.  The MapSig class decides how to pass the `context` argument depending on the signature of the function it is wrapping:
- If the function has a parameter named _'context'_, then MapSig will bind the `context` object to that parameter so that the function can modify the `context` content.
- If the function does not have a parameter named _'context'_, but does have a `VAR_KEYWORD` parameter (`**kwargs`), then MapSig will pass the contents of the `context` object as `**context` This will allow items within `context` to be bound to parameters in the function with matching names, but will prevent the `context` content from being modified.
- If the function does not have either a parameter named _'context'_, nor a `VAR_KEYWORD` parameter (`**kwargs`), then MapSig will **not** pass the contents of the `context` object to the function.


##### The order of positional and keyword arguments:
  for the following calling signature:
> `rule_method(item: SourceItem, event: TriggerEvent, *, context={})`


> **This is a valid signature:**
> 
> ```python
> def update_context(test_item, context, event: TriggerEvent):
>     pass
> ```
>
> - Starts with a positionally mapped `test_item` (to `item`), 
> followed by `event` and `context`, which map to keywords in `rule_method` 
> matching order is not required for keywords.
>


> **This is <u>not</u> a valid signature:**
> 
> ```python
> def update_context(event: TriggerEvent, test_item, context):
>     pass
> ```
>
> - `event` and `context` map to keywords in `rule_method`, 
> but `test_item` does not and is after the first mapped keyword (`event`).
>

#### Example rule method
The rule method is supplied at the the creation of a Rule object. The method is called on an item when that item passes the 'Test' defined for that Rule.

The Rule calls its method with the following arguments:
> `rule_method(item: SourceItem, event: TriggerEvent, *, context={})`

To keep Rule definitions clear and simple we want to be able to provide a function like `str.upper` as a rule method without having to define a custom wrapper like:
```Python
def my_upper(item: str, event: TriggerEvent, context)->str:
   return item.upper()
```

At the same time we want to be able to supply a function like:
```Python
def update_context(item: str, event: TriggerEvent, context):
    context['FoundItem'] = event.test_name
   return item
```
Although `context` is not returned explicitly, its contents are updated by he function, and a later processing function can use that context information.  For example:
```Python
def select_column(item: str,  **context):
    if context.get('FoundItem') == 'indent':
        return [None, item]
    else:
        return [item, None]
```


**Need to apply special context rules first**

1. If signature can be directly mapped from outer to inner, pass bound args & kwargs to inner.  No other work required.
2. Find keyword matches between inner & outer signatures and build keywords.
3. Map remaining positional arguments, dropping any missing ones.


### Defining MapSig

- __init__ defines calling_sig
- __call__ defines mapping process

In [28]:
class MapSig():
    context_parameter = 'context'
    wrapping_sig = inspect.Signature([
        inspect.Parameter('args', inspect.Parameter.VAR_POSITIONAL),
        inspect.Parameter('kwargs', inspect.Parameter.VAR_KEYWORD)
        ])

    @classmethod
    def set_sig(cls, *,
                 wrapping_func: Callable = None,
                 signature: inspect.Signature = None,
                 context_parameter: str = 'context') -> None:
        '''Define the calling signature for the wrapper.

        The signature is defined either by passing a function with the desired
        signature or by passing a Signature object.  If a wrapping_func is
        supplied, signature is ignored. if a wrapping signature is not defined
        `*args, **kwargs` is used.

        Updates:
            cls.wrapping_sig
            cls.context_parameter

        Args:
            wrapping_func (Callable, optional): An example function to use for
                defining the wrapping signature.
            signature (inspect.Signature, optional): A Signature object
                defining the wrapping signature.  Ignored if wrapping_func is
                    supplied.
            context_parameter (str, optional):  The name of the parameter in the
                wrapping signature to  identify as the special context parameter
                that can act as both  an object or a **kwargs parameter.
                Defaults to 'context'.
        '''
        if wrapping_func:
            cls.wrapping_sig = inspect.signature(wrapping_func)
        elif signature:
            cls.wrapping_sig = signature
        if context_parameter:
            cls.context_parameter = context_parameter

    @staticmethod
    def is_keyword(param: inspect.Parameter) -> bool:
        '''Return True if param can be a Keyword argument.'''
        return param.kind.name in ['KEYWORD_ONLY', 'POSITIONAL_OR_KEYWORD']

    @staticmethod
    def is_positional(param: inspect.Parameter) -> bool:
        '''Return True if param can be a positional argument.'''
        return 'POSITIONAL_' in param.kind.name

    @property
    def wrapping_keywords(self) -> Tuple[str]:
        '''The names of all parameters in the wrapping signature which may be
        keywords.
        '''
        keywords = tuple(
            param.name for param in self.wrapped_sig.parameters.values()
            if self.is_keyword(param))
        return keywords

    @property
    def __signature__(self) -> inspect.Signature:
        '''The Signature comes from the class defined wrapping signature.'''
        return self.wrapping_sig

    @property
    def __annotations__(self) -> Dict[str, Any]:
        '''The annotations are derived by combining the class defined wrapping
        signature with the return type from the wrapped function.'''
        annotation = {}
        for param in self.wrapping_sig.parameters.values():
            if not param.annotation is param.empty:
                name = param.name
                ant_type = param.annotation
            annotation[name] = ant_type
        if not self.wrapped_sig.return_annotation is self.wrapped_sig.empty:
            annotation['return'] = self.wrapped_sig.return_annotation
        return annotation

    @property
    def has_kwarg(self):
        '''Return True if the wrapped function has a VAR_KEYWORD (**kwargs)
        parameter.'''
        param_kinds = set(param.kind.name
                          for param in self.wrapped_sig.parameters.values())
        return 'VAR_KEYWORD' in param_kinds

    @property
    def mapped_keyword_names(self) -> Tuple[str]:
        '''The names of all arguments that have been successfully mapped to
        keywords in the wrapped function.
        '''
        return tuple(self.mapped_keywords.keys())

    def reset(self) -> None:
        '''Initialize calling arguments.'''
        self.bound_args = None
        self.mapped_keywords = {}
        self.mapped_args = tuple()

    def __init__(self, wrapped_func: Callable) -> None:
        '''Wrap a function with a class defined function signature.

        Class objects are a wrapper function around the supplied wrapped
        function.

        Args:
            wrapped_func (Callable): The function to be wrapped with a
            pre-defined signature.
        '''
        # Initialize wrapped function
        # Store the function to be wrapped and identify it's signature so that
        # when called the appropriate arguments will be passed.
        self.wrapped = wrapped_func
        self.wrapped_sig = inspect.signature(wrapped_func)
        # The names of all parameters in the wrapped function that may be keywords.
        self.wrapped_keywords = tuple(
            param.name for param in self.wrapped_sig.parameters.values()
            if self.is_keyword(param))
        # Initialize calling arguments
        # bound_args, mapped_keywords and mapped_args are instance attributes
        # that link arguments passed to the wrapping function with arguments
        # for the wrapped function.
        self.bound_args = None
        self.mapped_keywords = {}
        self.mapped_args = tuple()
        # Update annotations
        # The annotations should reflect the class-defined signature, but the
        # remaining introspection from the wrapped function.
        to_update = ('__module__', '__name__', '__qualname__', '__doc__',
                     '__type_params__')
        functools.update_wrapper(self, wrapped_func, assigned=to_update)
        # Update the wrapping signature with the return annotation from the
        # wrapped function.
        wrapped_ra = self.wrapped_sig.return_annotation
        new_sig = self.wrapping_sig.replace(return_annotation=wrapped_ra)
        self.wrapping_sig = new_sig
        #self.__call__.__annotations__ = self.__annotations__
        #self.__call__.__doc__ = self.wrapped.__doc__

    def is_matched_args(self)->bool:
        '''Map the calling arguments directly to the wrapped function.

        If a direct mapping of the arguments is possible, return True and set
        the mapped_args and mapped_keywords attributes accordingly.

        Updates:
            self.mapped_args
            self.mapped_keywords

        Returns:
            bool: True if a direct mapping of the arguments is possible.
        '''
        try:
            bound_wrapped = self.wrapped_sig.bind(*self.bound_args.args,
                                                  **self.bound_args.kwargs)
        except TypeError:
            return False
        self.mapped_args = bound_wrapped.args
        self.mapped_keywords = bound_wrapped.kwargs
        return True

    def get_keywords(self):
        '''Link argument names in bound_sig with parameters in the target_sig

        Generate a dictionary of KEYWORD parameters and corresponding bound
        arguments for all parameters from target_sig that match the  name of a
        bound argument in bound_sig.

        Updates:
            self.mapped_keywords
        '''
        keywords = {}
        for name in self.wrapped_keywords:
            if name in self.bound_args.arguments:
                keywords[name] = self.bound_args.arguments[name]
        self.mapped_keywords = keywords

    def get_positional(self) -> None:
        '''Build a tuple of positional arguments for calling the wrapped
        function.

        If the argument is a valid positional argument, match the argument
        with the corresponding positional parameter in the wrapped function.

        Stop matching positional arguments at the first mapped keyword argument.

        Updates:
            self.mapped_args
        '''
        position_map = zip(self.wrapping_sig.parameters.values(),
                           self.wrapped_sig.parameters.values())
        positional_args = []
        for in_p, out_p in position_map:
            # Stop matching positional arguments at the first mapped keyword
            # argument.
            is_mapped = ((in_p in self.mapped_keyword_names) |
                         (out_p in self.mapped_keyword_names))
            if is_mapped:
                break
            # If the argument is a valid positional argument, match the
            # argument with the corresponding positional parameter in the
            # wrapped function.
            if self.is_positional(in_p) & self.is_positional(out_p):
                positional_args.append(self.bound_args.arguments[in_p.name])
        # Build a tuple of positional arguments for calling teh wrapped
        # function.
        self.mapped_args = tuple(positional_args)

    def __call__(self, *args, **kwargs) -> Any:
        '''Call the wrapped function.
        '''
        self.reset()
        try:
            self.bound_args = self.wrapping_sig.bind(*args, **kwargs)
        except TypeError as err:
            raise InvalidArgumentsError() from err
        if self.is_matched_args():
            return self.wrapped(*args, **kwargs)
        self.get_keywords()
        if self.context_parameter not in self.mapped_keyword_names:
            if self.has_kwarg:
                context = self.bound_args.arguments.get('context', {})
                self.mapped_keywords.update(context)
        self.get_positional()
        return self.wrapped(*self.mapped_args, **self.mapped_keywords)


### Sub-classing MapSig

In [29]:
def rule_method(item: str, event: TriggerEvent, *, context: Context = {})->str:
   pass

calling_sig = inspect.signature(rule_method)
print('Rule Method Signature')
print_param_list(calling_sig)


Rule Method Signature
Name            Kind                     Default     Annotation               
item            POSITIONAL_OR_KEYWORD                <class 'str'>            
event           POSITIONAL_OR_KEYWORD                <class 'sections.TriggerEvent'>
context         KEYWORD_ONLY             {}          typing.Dict[str, typing.Any]


In [30]:
class RuleSig(MapSig):
    wrapping_sig = inspect.Signature([
        inspect.Parameter('item', inspect.Parameter.POSITIONAL_OR_KEYWORD,
                          annotation=str),
        inspect.Parameter('event', inspect.Parameter.POSITIONAL_OR_KEYWORD,
                          annotation=TriggerEvent),
        inspect.Parameter('context', inspect.Parameter.KEYWORD_ONLY,
                          annotation=Context)
        ])


### MapSig Tests
- **Without** `**kwargs`
  - Exactly matching signatures
  - Dropped positional from right
  - Dropped positional from left (using keyword match)
  - Changed order of keywords
  - `context` as Object
  - partial functions
- **With** `**kwargs`
  - `context` as **kwargs 
  - Keywords in `context`
- Incompatible functions
    - Too many positional parameters? 
- *args in wrapped function ?
- *args in wrapping function ?

#### Exactly matching signatures

In [31]:
example_item = 'Test Item'
dummy_event = TriggerEvent()
context = {'ContextItem': 'Item Value'}


@RuleSig
def matched_sig_func(item, event, context):
    args = {
        'item': item,
        'event': event,
        'context': context
        }
    pprint(args)
    return item


matched_sig_func(item=example_item, event=dummy_event, context=context)


{'context': {'ContextItem': 'Item Value'},
 'event': <sections.TriggerEvent object at 0x00000201E6B21AF0>,
 'item': 'Test Item'}


'Test Item'

#### Dropped positional from right

In [32]:
@RuleSig
def dropped_event_func(test_item) -> str:
    args = {'item': test_item}
    pprint(args)
    return test_item


dropped_event_func(item=example_item, event=dummy_event, context=context)

{'item': 'Test Item'}


'Test Item'

#### Dropped positional from left (using keyword match)

In [33]:
@RuleSig
def dropped_event_func(event) -> str:
    args = {'event': event}
    pprint(args)
    return event


dropped_event_func(item=example_item, event=dummy_event, context=context)

TypeError: dropped_event_func() got multiple values for argument 'event'

# Done To Here

# TODO Fix annotations for RuleSig.__call__

In [39]:
dropped_event_func.__annotations__

{'item': str,
 'event': sections.TriggerEvent,
 'context': typing.Dict[str, typing.Any],
 'return': str}

In [40]:
dropped_event_func.__call__.__annotations__

{'return': typing.Any}

In [41]:
wrapped_sig = inspect.signature(dropped_event_func)
str(wrapped_sig)

'(item: str, event: sections.TriggerEvent, *, context: Dict[str, Any]) -> str'

In [42]:
wrapped_sig = inspect.signature(dropped_event_func.__call__)
str(wrapped_sig)

'(*args, **kwargs) -> Any'

In [ ]:
@RuleSig
def matched_sig_func(item: str, event: TriggerEvent, context) -> str:
    '''This function has the exact same signature as te wrapping function.

    Args:
        item (str): An item
        event (TriggerEvent): An event
        context (Any): The context

    Returns:
        str: The item
    '''
    args = {
        'item': item,
        'event': event,
        'context': context
        }
    pprint(args)
    return item

In [ ]:
example_item = 'Paris'

dummy_event = TriggerEvent()
dummy_event.trigger_name = 'Place'
dummy_event.test_passed = True
dummy_event.test_name = 'Paris'
dummy_event.test_value = 'Paris'

context = {
   'Break': 'SubSectionStart',
   'Current Section': 'Sub-Section',
   'Event': 'StartSection',
   'Skipped Lines': ['Text between sections'],
   'Status': 'End of Source'
   }

In [ ]:
matched_sig_func.__annotations__

{'item': str,
 'event': sections.TriggerEvent,
 'context': typing.Dict[str, typing.Any],
 'return': str}

In [ ]:
wrapped_sig = inspect.signature(matched_sig_func)
str(wrapped_sig)

'(item: str, event: sections.TriggerEvent, *, context: Dict[str, Any]) -> str'

In [ ]:
matched_sig_func.__name__

'matched_sig_func'

In [ ]:
matched_sig_func.__qualname__

'matched_sig_func'

In [ ]:
print(matched_sig_func.__doc__)

This function has the exact same signature as te wrapping function.

    Args:
        item (str): An item
        event (TriggerEvent): An event
        context (Any): The context

    Returns:
        str: The item
    


In [ ]:
print('Rule Method Signature')
print_sig_info(calling_sig)

In [ ]:
context

{'Break': 'SubSectionStart',
 'Current Section': 'Sub-Section',
 'Event': 'StartSection',
 'Skipped Lines': ['Text between sections'],
 'Status': 'End of Source',
 'FoundItem': 'Paris'}

In [9]:
def rule_method(item: str, event: TriggerEvent, *, context: Context = {})->str:
   pass

calling_sig = inspect.signature(rule_method)
print('Rule Method Signature')
print_sig_info(calling_sig)


Rule Method Signature
item	positional or keyword    
event	positional or keyword    
context	keyword-only             {}


In [10]:
bound_sig = calling_sig.bind(item=example_item, event=dummy_event, context=context)
bound_sig.arguments


{'item': 'Paris',
 'event': <sections.TriggerEvent at 0x19f13ad5e80>,
 'context': {'Break': 'SubSectionStart',
  'Current Section': 'Sub-Section',
  'Event': 'StartSection',
  'Skipped Lines': ['Text between sections'],
  'Status': 'End of Source'}}

In [11]:
bound_sig.apply_defaults()
bound_sig.arguments

{'item': 'Paris',
 'event': <sections.TriggerEvent at 0x19f13ad5e80>,
 'context': {'Break': 'SubSectionStart',
  'Current Section': 'Sub-Section',
  'Event': 'StartSection',
  'Skipped Lines': ['Text between sections'],
  'Status': 'End of Source'}}

### Matching Signatures

In [12]:
def update_context(item: str, event: TriggerEvent, context):
    context['FoundItem'] = event.test_name
    return item

In [13]:
print('\nupdate_context Signature')
context_sig = inspect.signature(update_context)
print_sig_info(context_sig)


update_context Signature
item	positional or keyword    
event	positional or keyword    
context	positional or keyword    


In [14]:
try:
    ctx_bnd = context_sig.bind(*bound_sig.args, **bound_sig.kwargs)
except TypeError:
    print('Arguments do not match')
else:
    print(ctx_bnd)

<BoundArguments (item='Paris', event=<sections.TriggerEvent object at 0x0000019F13AD5E80>, context={'Break': 'SubSectionStart', 'Current Section': 'Sub-Section', 'Event': 'StartSection', 'Skipped Lines': ['Text between sections'], 'Status': 'End of Source'})>


In [15]:
update_context(*bound_sig.args, **bound_sig.kwargs)
pprint(context)

{'Break': 'SubSectionStart',
 'Current Section': 'Sub-Section',
 'Event': 'StartSection',
 'FoundItem': 'Paris',
 'Skipped Lines': ['Text between sections'],
 'Status': 'End of Source'}


### Keyword Matching

In [16]:
def no_item_context(context, event: TriggerEvent):
    context['FoundEvent'] = event.test_name
    return event.trigger_name


ctx_no_item_sig = inspect.signature(no_item_context)
print_sig_info(ctx_no_item_sig)

context	positional or keyword    
event	positional or keyword    


In [17]:
kwds = get_keywords(bound_sig, ctx_no_item_sig)
kwds

{'context': {'Break': 'SubSectionStart',
  'Current Section': 'Sub-Section',
  'Event': 'StartSection',
  'Skipped Lines': ['Text between sections'],
  'Status': 'End of Source',
  'FoundItem': 'Paris'},
 'event': <sections.TriggerEvent at 0x19f13ad5e80>}

In [18]:
no_item_context(**kwds)
pprint(context)

{'Break': 'SubSectionStart',
 'Current Section': 'Sub-Section',
 'Event': 'StartSection',
 'FoundEvent': 'Paris',
 'FoundItem': 'Paris',
 'Skipped Lines': ['Text between sections'],
 'Status': 'End of Source'}


### Positional matching

In [19]:
upper_sig = inspect.signature(str.upper)
print_sig_info(upper_sig)


self	positional-only          


In [20]:
mapped_args = get_positional(bound_sig, upper_sig, calling_sig)
mapped_args

('Paris',)

In [21]:
str.upper(*mapped_args)

'PARIS'

In [22]:
kwds = get_keywords(bound_sig, upper_sig)
kwds

{}

In [23]:
mapped_args = get_positional(bound_sig, upper_sig, calling_sig, kwds)
mapped_args

('Paris',)

In [24]:
try:
    ctx_bnd = upper_sig.bind(*bound_sig.args, **bound_sig.kwargs)
except TypeError:
    print('Arguments do not match')


Arguments do not match


### Single Keyword

In [25]:
def event_name(event: TriggerEvent):
    return event.trigger_name

event_sig = inspect.signature(event_name)
print_sig_info(event_sig)

event	positional or keyword    


In [26]:
try:
    ctx_bnd = event_sig.bind(*bound_sig.args, **bound_sig.kwargs)
except TypeError:
    print('Arguments do not match')


Arguments do not match


In [27]:
kwds = get_keywords(bound_sig, event_sig)
kwds

{'event': <sections.TriggerEvent at 0x19f13ad5e80>}

In [28]:
mapped_args = get_positional(bound_sig, event_sig, calling_sig, kwds)
mapped_args

()

In [29]:
bound_sig

<BoundArguments (item='Paris', event=<sections.TriggerEvent object at 0x0000019F13AD5E80>, context={'Break': 'SubSectionStart', 'Current Section': 'Sub-Section', 'Event': 'StartSection', 'Skipped Lines': ['Text between sections'], 'Status': 'End of Source', 'FoundItem': 'Paris', 'FoundEvent': 'Paris'})>

In [30]:
event_sig

<Signature (event: sections.TriggerEvent)>

In [31]:
event_name(*mapped_args, **kwds)

'Place'

### **context

#### The `context` parameter.
- If the function has a parameter named _'context'_:
  - Bind the `context` object to that parameter .
- If the function does not have a parameter named _'context'_, but does have a `VAR_KEYWORD` parameter (`**kwargs`):
  - Pass the contents of the `context` object as `**context`.
- If the function does not have either a parameter named _'context'_, nor a `VAR_KEYWORD` parameter (`**kwargs`):, then MapSig will
  - Do not pass the contents of the `context` object to the function.

In [32]:
def select_column(item: str,  **context):
    if context.get('FoundItem') == 'indent':
        return [None, item]
    else:
        return [item, None]

context['FoundItem'] = 'indent'
col_sig = inspect.signature(select_column)
print_sig_info(col_sig)

item	positional or keyword    
context	variadic keyword         


In [33]:
try:
    ctx_bnd = col_sig.bind(*bound_sig.args, **bound_sig.kwargs)
except TypeError:
    print('Arguments do not match')


Arguments do not match


In [34]:
kwds = get_keywords(bound_sig, col_sig)
kwds

{'item': 'Paris'}

In [35]:
if 'context' not in kwds:
    pass


'VAR_KEYWORD' in [param.kind.name for param in col_sig.parameters.values()]

True

In [36]:
ctx = bound_sig.arguments.get('context', {})
kwds.update(ctx)
kwds

{'item': 'Paris',
 'Break': 'SubSectionStart',
 'Current Section': 'Sub-Section',
 'Event': 'StartSection',
 'Skipped Lines': ['Text between sections'],
 'Status': 'End of Source',
 'FoundItem': 'indent',
 'FoundEvent': 'Paris'}

In [37]:
mapped_args = get_positional(bound_sig, col_sig, calling_sig, kwds)
mapped_args

()

In [38]:
select_column(*mapped_args, **kwds)

[None, 'Paris']

In [46]:
event_sig = inspect.signature(update_context)
print_sig_info(event_sig)

item	positional or keyword    
event	positional or keyword    
context	keyword-only             


In [47]:
update_context.__annotations__

{'item': str, 'event': sections.TriggerEvent}

In [48]:
def foo(a, /, b: str, *c, d: Any, e: int = 10, **f) -> str:
    return b

foo_sig = inspect.signature(foo)
print_sig_info(foo_sig)
print(str(foo_sig))
print_param_list(foo_sig)
foo.__annotations__

a	positional-only          
b	positional or keyword    
c	variadic positional      
d	keyword-only             
e	keyword-only             10
f	variadic keyword         
(a, /, b: str, *c, d: Any, e: int = 10, **f) -> str
Name            Kind                     Default     Annotation               
a               POSITIONAL_ONLY                      <class 'inspect._empty'> 
b               POSITIONAL_OR_KEYWORD                <class 'str'>            
c               VAR_POSITIONAL                       <class 'inspect._empty'> 
d               KEYWORD_ONLY                         typing.Any               
e               KEYWORD_ONLY             10          <class 'int'>            
f               VAR_KEYWORD                          <class 'inspect._empty'> 


{'b': str, 'd': typing.Any, 'e': int, 'return': str}

`functools.update_wrapper(wrapper, wrapped, assigned=WRAPPER_ASSIGNMENTS, updated=WRAPPER_UPDATES)`
> Update a wrapper function to look like the wrapped function. The optional arguments are tuples to specify which attributes of the original function are assigned directly to the matching attributes on the wrapper function and which attributes of the wrapper function are updated with the corresponding attributes from the original function. The default values for these arguments are the module level constants WRAPPER_ASSIGNMENTS (which assigns to the wrapper function’s `__module__`, `__name__`, `__qualname__`, `__annotations__` and `__doc__`, the documentation string) and WRAPPER_UPDATES (which updates the wrapper function’s `__dict__`, i.e. the instance dictionary).
>
> To allow access to the original function for introspection and other purposes (e.g. bypassing a caching decorator such as lru_cache()), this function automatically adds a `__wrapped__` attribute to the wrapper that refers to the function being wrapped.
>
> The main intended use for this function is in decorator functions which wrap the decorated function and return the wrapper. If the wrapper function is not updated, the metadata of the returned function will reflect the wrapper definition rather than the original function definition, which is typically less than helpful.
>
> `update_wrapper()` may be used with callables other than functions. Any attributes named in assigned or updated that are missing from the object being wrapped are ignored (i.e. this function will not attempt to set them on the wrapper function). `AttributeError` is still raised if the wrapper function itself is missing any attributes named in updated.


In [49]:
functools.WRAPPER_ASSIGNMENTS

('__module__',
 '__name__',
 '__qualname__',
 '__doc__',
 '__annotations__',
 '__type_params__')

In [50]:
functools.WRAPPER_UPDATES

('__dict__',)

In [51]:
def build_annotation(sig):
    annotation = {}
    for param in sig.parameters.values():
        if not param.annotation is param.empty:
            name = param.name
            ant_type = param.annotation
        annotation[name] = ant_type
    if not sig.return_annotation is sig.empty:
        annotation['return'] = sig.return_annotation
    return annotation

In [52]:
def no_item_context(context, event: TriggerEvent) -> str:
    context['FoundEvent'] = event.test_name
    return event.trigger_name


ctx_no_item_sig = inspect.signature(no_item_context)
print_sig_info(ctx_no_item_sig)
print(str(ctx_no_item_sig))
print_param_list(ctx_no_item_sig)
no_item_context.__annotations__

context	positional or keyword    
event	positional or keyword    
(context, event: sections.TriggerEvent) -> str
Name            Kind                     Default     Annotation               
context         POSITIONAL_OR_KEYWORD                <class 'inspect._empty'> 
event           POSITIONAL_OR_KEYWORD                <class 'sections.TriggerEvent'>


{'event': sections.TriggerEvent, 'return': str}

In [88]:
wrapped_update_context = RuleSig(update_context)

In [ ]:
wrapped_update_context(example_item, dummy_event, context)

### MapSig annotations

In [ ]:
def select_column(item: str,  **context):
    if context.get('FoundItem') == 'indent':
        return [None, item]
    else:
        return [item, None]

context['FoundItem'] = 'indent'
col_sig = inspect.signature(select_column)
print_sig_info(col_sig)

In [38]:
inspect.get_annotations(select_column)

{'item': str, 'return': typing.List[str]}

In [40]:
select_column.__annotations__

{'item': str, 'return': typing.List[str]}

In [39]:
select_column.__doc__

'Put string in teh correct column\n\n    Args:\n        item (str): the sting to add\n\n    Returns:\n        List[str]: column_row\n    '

In [ ]:
mapped_args = get_positional(bound_sig, col_sig, calling_sig, kwds)
mapped_args

()

In [61]:
def foo(a, /, b: str, *c, d: Any, e: int = 10, **f) -> str:
    return b

sig = inspect.signature(foo)
print_param_list(sig)

Name            Kind                     Default     Annotation               
a               POSITIONAL_ONLY                      <class 'inspect._empty'> 
b               POSITIONAL_OR_KEYWORD                <class 'str'>            
c               VAR_POSITIONAL                       <class 'inspect._empty'> 
d               KEYWORD_ONLY                         typing.Any               
e               KEYWORD_ONLY             10          <class 'int'>            
f               VAR_KEYWORD                          <class 'inspect._empty'> 


In [62]:
class RuleSig(MapSig):
    default_signature = inspect.Signature([
        inspect.Parameter('item', inspect.Parameter.POSITIONAL_OR_KEYWORD,
                          annotation=str),
        inspect.Parameter('event', inspect.Parameter.POSITIONAL_OR_KEYWORD,
                          annotation=TriggerEvent),
        inspect.Parameter('context', inspect.Parameter.KEYWORD_ONLY,
                          annotation=Context)
        ])
    wrapping_sig = default_signature.replace()

In [63]:
wrapped_foo = RuleSig(foo)

In [64]:
sig = inspect.signature(wrapped_foo)
print_param_list(sig)


Name            Kind                     Default     Annotation               
item            POSITIONAL_OR_KEYWORD                <class 'str'>            
event           POSITIONAL_OR_KEYWORD                <class 'sections.TriggerEvent'>
context         KEYWORD_ONLY                         typing.Dict[str, typing.Any]


In [65]:
wrapped_foo.__annotations__

{'b': str, 'd': typing.Any, 'e': int, 'return': str}

In [ ]:
select_column(*mapped_args, **kwds)

[None, 'Paris']

In [47]:
Counter(param.kind.name for param in col_sig.parameters.values())

Counter({'POSITIONAL_OR_KEYWORD': 1, 'VAR_KEYWORD': 1})

In [47]:
Counter(param.kind.name for param in upper_sig.parameters.values())

Counter({'POSITIONAL_ONLY': 1})

In [40]:
calling_sig.parameters.keys()

odict_keys(['item', 'event', 'context'])

In [ ]:
ctx_bnd.apply_defaults()

pprint(ctx_bnd.arguments)
print('\n')


pprint(context)

- rule_method(item, context) is not allowed; use:
    - rule_method(item, **context)          or
    - rule_method(item, event, context)     instead.

*Required signature:*
> 

|Signature                                 |Args|varkw  |Notes              |
|------------------------------------------|----|-------|-------------------|
|func(item)                                |1   |None   |                   |
|func(item, **context)                     |1   |context|                   |
|func(item, event)                         |2   |None   |                   |
|func(item, event [, other(s)=defaults(s)])|1   |None   |                   |
|func(item, event, **context)              |2   |context|                   |
|func(item, event, context)                |3   |None   |Expected           |
|func(item, event, [other(s),] **context)  |3+  |context|Not Yet Implemented|

**Valid Action Names**
- 'Original': return the original item supplied.
- 'Blank': return ''  (an empty string).
- 'None': return None.
- 'Value': return the self.event.test_value object.
- 'Name': return the self.event.test_name object.

#### Allow processing functions to take specific keyword arguments from context

add a wrapper to trap unwanted context items

1. Get parameters and defaults for all arguments of the
   supplied function.
2. Generate a defaults dictionary for all keyword
   arguments.
3. When called, pass non-keyword arguments directly to
   the original function.
4. Update the defaults dictionary with calling
   keywords except the context keyword, if present.
5. If context is among the default keyword arguments
   and is supplied when called, add context to the keyword dictionary; otherwise search the supplied context for any keys in the defaults dictionary and use them to update the defaults dictionary.
10. If a **kwargs is present in the original function,
    pass all remaining context items using **context.



### Initialization
A MapSig object is defined by passing a dummy function that has teh desired signature for the final wrapped function.  e.g.

```python
def dummy_func(arg1: str, arg2='', **arg3)->str: pass

three_arg_wrapper = MapSig(dummy_func)

@three_arg_wrapper
def one_string_func(string1: str)->str:
    return string1.lower()

one_string_func('ABC', 'DEF', extra_arg='Nothing')
>>> 'abc'
```

### Mapping parameters between signatures.

The "Calling" signature is defined with sequence of argument definitions. 

**Argument Definition:**
- **Name** (str): The name of the argument.
- **Type** (str): _'Positional'_, _'Keyword'_, _'kwarg'_
  - All _'Positional'_ arguments must precede the first _'Keyword'_ argument.
  - A _'kwarg'_ argument is not required, but there can only be one _'kwarg'_ argument, and if present, it must be the last argument.
  - If the "Calling" signature includes a _'kwarg'_ argument and the un-wrapped function signature contains a parameter name matching the _'kwarg'_ argument, the _'kwarg'_ argument will be passed as a dictionary object to the the un-wrapped function.
  - If the "Calling" signature includes a _'kwarg'_ argument and the un-wrapped function signature **does not** contain a parameter name matching the _'kwarg'_ argument, but does contain it's own `**kwargs` type parameter, the _'kwarg'_ argument the _'kwarg'_ argument will be passed to the the un-wrapped function in the `**kwarg` style.
  - The _'kwarg'_ argument is expected to have the `Dict[str, Any]` data type so that it can be passed to the un-wrapped function in the `**kwarg` style if required.
- **Required** (bool, optional): If an argument is required, there must be a corresponding argument in the un-wrapped function signature.
  - Default is False.
  - Any _'Keyword'_ argument can be required regardless of order.
  - The "Calling" signature must begin with any required _'Positional'_ arguments. (A required _'Positional'_ argument may not follow any non-required _'Positional'_ arguments.)

#### "Calling" signature checks
- if Type is not one of [_'Positional'_, _'Keyword'_, _'kwarg'_], `ValueError` is raised.
- If a _'Positional'_ argument follows the first _'Keyword'_ argument, `ValueError` is raised.
- If a "required" _'Positional'_ argument follows a "non-required" _'Positional'_ argument, `ValueError` is raised.

### \_\_call__ method
- The `__call__(func)` method is used to wrap `func`.
- It compares the `func` signature with the defined "Calling" signature

1. _'Keyword'_ arguments are matched with `func` _'positional or keyword'_ and  _'keyword-only'_ parameters that have the same name.

2. If the `func` signature **does not** contain a 'variadic keyword' (`**kwargs`) parameter, all _'Keyword'_ arguments **following** the first matched _'Keyword'_ argument are not passed to `func`. If any of those arguments is 'Required' a `ValueError` is raised.

3. A sequence is created from the _'Positional'_ arguments and all arguments **preceding** the first matched _'Keyword'_ argument.
  
4. A second sequence is created from the _'positional-only'_ and _'positional or keyword'_ `func` parameters.

5. The two sequences are mapped 1-to-1 until one of the sequences is exhausted. 

6. If the second sequence (containing `func` parameters) is exhausted first, and the `func` signature **does not** contain a 'variadic positional' (`*args`) parameter, any remaining _'Positional'_ arguments are not passed to `func`.  If any of those arguments is 'Required' a `ValueError` is raised.

- defaults
- `*args`
- `**kwargs`
- The _'kwarg'_ argument


1. If the `func` signature **contains** a 'variadic keyword' (`**kwargs`), all un matched _'Keyword'_ arguments are passed to `func` via the `**kwargs` assignment.



  
> - Without a list of valid signatures it will assume that positional arguments 
>   come in the same order as in the final signature.
> - Additional keyword arguments can be extracted from the 'varkw' argument 
>   (if it exists) in the final signature.
> - Additional keyword arguments can be supplied during instance creation and passed
>   to the called function.

Move standard_action, sig_match, and set_method into the module

## Classes as function wrappers
- Class objects are decorators

- Action class
  - Convert *Action* strings into standard functions

- StandardSig class
  - Check the function argument signature against expected possible signatures.
  - Wrap the function with a standard argument signature.

- FunctionProcessor Class
  - Apply specific Action and StandardSig instances
  - Add attribute indicating if the supplied function is a generator.

### StandardSig Class 
  - Check the function argument signature against expected possible signatures.
  - Wrap the function with a standard argument signature.
  - Assign annotation to resulting function based on final signature and wrapped function.

A sequence of variable definitions is given representing the desired final signature.

has __call__ method 



Definition
- Final (required) signature 
- Valid signatures
  
> - Without a list of valid signatures it will assume that positional arguments 
>   come in the same order as in the final signature.
> - Additional keyword arguments can be extracted from the 'varkw' argument 
>   (if it exists) in the final signature.
> - Additional keyword arguments can be supplied during instance creation and passed
>   to the called function.

##### Example:

**Required signature:**
> `rule_method(test_object: SourceItem, event: TriggerEvent, context)`

|Name: str  |Type: [type, str]|Required: bool  |Positional: bool|Notes: str|
|-----------|-----------------|----------------|----------------|-----------|
|test_object|SourceItem       |True            |True            |Must be present, cannot be in context|
|event      |TriggerEvent     |False           |True            |can be in context?|
|context    |'varkw'          |False           |True            |context  or **context|


**Valid signatures**

The conversion is based on number of arguments without defaults and the
presence of a var keyword argument (**kwargs):


|Signature                                 |Args|varkw  |Notes              |
|------------------------------------------|----|-------|-------------------|
|func(item)                                |1   |None   |                   |
|func(item, **context)                     |1   |context|                   |
|func(item, event)                         |2   |None   |                   |
|func(item, event [, other(s)=defaults(s)])|1   |None   |                   |
|func(item, event, **context)              |2   |context|                   |
|func(item, event, context)                |3   |None   |Expected           |
|func(item, event, [other(s),] **context)  |3+  |context|Not Yet Implemented|


**Valid Action Names**
- 'Original': return the original item supplied.
- 'Blank': return ''  (an empty string).
- 'None': return None.
- 'Value': return the self.event.test_value object.
- 'Name': return the self.event.test_name object.

```python
rule_sig = {
    (1, False): lambda func, item, event, context: func(item),
    (1, True): lambda func, item, event, context: func(item, **context),
    (2, False): lambda func, item, event, context: func(item, event),
    (2, True): lambda func, item, event, context: func(item, event, **context),
    (3, False): lambda func, item, event, context: func(item, event, context)
    }
```

### FunctionProcessor Class

- Class objects are decorators / functions that 
  1. Convert *Action* strings into standard functions
  3. Check the function argument signature against expected possible signatures.
  4. Wrap the function with a standard argument signature.
  5. Test whether a supplied function is a generator.
  


- Take text processing functions and convert them into *actions*

- Convert set_method, sig_match, standard_action into a helper class

The class can be initialized with signature and action definitions to create different objects for different function groups. When the object is applied to a function or string, it returns a corresponding function with the expected argument signature.

The class object is a custom decorator function.

https://stackoverflow.com/questions/76304643/reuse-of-function-signature-type-hints-and-intellisense-args-kwargs


### Standard Actions
*Convert a Method name to a Standard Function.*

Take the name of a standard actions and return the matching function.

**Arguments:**
- action_name (str): The name of a standard action.
- method_type (str): The type of action function expected.
    One of ['Process', 'Rule']. Defaults to 'Process'

**Raises: **
- ValueError If the action_name or method_type supplied are not one of
    the valid action names or method types.

**Returns:**
- (ProcessFunc, RuleFunc): One of the standard action functions.

### sig_match
*Convert the supplied function with a standard signature.*

The conversion is based on number of arguments without defaults and the
presence of a var keyword argument (**kwargs):

**Arguments:**
- given_method (MethodOptions): A function to be used as a rule
        method or a process method.
- sig_type (str, optional): The type of argument signature desired.  Can
        be one of: 'Process' or 'Rule'.  Defaults to 'Process'.

**Raises:**
- ValueError If given_method does not have one of the expected
    argument signature types.

**Returns:**
- (ProcessFunc, RuleFunc): A function with the standard Rule or Process
        Method argument signature


### set_method
*Convert the supplied function or action name to a Function with
    the standard signature.*

See the standard_action function for valid action names. See the sig_match
function for valid function argument signatures.
Add a special attribute to the returned function indicating if it is a
generator function.

**Arguments:**
- given_method (RuleMethodOptions): A function, or the name of a
    standard action.
- method_type (str): The type of action function expected.
    One of ['Process', 'Rule']. Defaults to 'Process'

**Raises:**
- ValueError If rule_method is a string and is not one of the
    valid action names, or if rule_method is a function and does not
    have one of the valid argument signature types.

**Returns:**
- (ProcessFunc, RuleFunc): A function with the standard Rule or Process
        Method argument signature

### Method Types

#### rule
- rule_method(item, context) is not allowed; use:
    - rule_method(item, **context)          or
    - rule_method(item, event, context)     instead.

*Required signature:*
> `rule_method(test_object: SourceItem, event: TriggerEvent, context)`

|Signature                                 |Args|varkw  |Notes              |
|------------------------------------------|----|-------|-------------------|
|func(item)                                |1   |None   |                   |
|func(item, **context)                     |1   |context|                   |
|func(item, event)                         |2   |None   |                   |
|func(item, event [, other(s)=defaults(s)])|1   |None   |                   |
|func(item, event, **context)              |2   |context|                   |
|func(item, event, context)                |3   |None   |Expected           |
|func(item, event, [other(s),] **context)  |3+  |context|Not Yet Implemented|

**Valid Action Names**
- 'Original': return the original item supplied.
- 'Blank': return ''  (an empty string).
- 'None': return None.
- 'Value': return the self.event.test_value object.
- 'Name': return the self.event.test_name object.

#### process
- Also applies to Sentinel and Assemble methods.

*Required signature:*
> `process_method(test_object: SourceItem, context)`

|Signature                                 |Args|varkw  |Notes              |
|------------------------------------------|----|-------|-------------------|
|func(item)                                |1   |None   |                   |
|func(item  [, other(s)=defaults(s)])      |1   |None   |                   |
|func(item, **context)                     |1   |context|                   |
|func(item, context)                       |2   |None   |Expected           |
|func(item  [, other(s)] **context)        |2+  |context|Not Yet Implemented|

**Valid Action Names**
- 'Original': return the original item supplied.
- 'Blank': return ''  (an empty string).
- 'None': return None.



#### Function Signature options

In [6]:
[k.description for k in inspect._ParameterKind.__members__.values()]

['positional-only',
 'positional or keyword',
 'variadic positional',
 'keyword-only',
 'variadic keyword']

In [10]:
[k.name for k in inspect._ParameterKind.__members__.values()]

['POSITIONAL_ONLY',
 'POSITIONAL_OR_KEYWORD',
 'VAR_POSITIONAL',
 'KEYWORD_ONLY',
 'VAR_KEYWORD']

**kind**
> Describes how argument values are bound to the parameter. The possible values are accessible via Parameter (like Parameter.KEYWORD_ONLY), and support comparison and ordering, in the following order:

|Name|Description|Meaning|
|----|-----------|-------|
POSITIONAL_ONLY|`'positional-only'`|Value must be supplied as a positional argument. Positional only parameters are those which appear before a / entry (if present) in a Python function definition.|
|POSITIONAL_OR_KEYWORD|`'positional or keyword'`|Value may be supplied as either a keyword or positional argument (this is the standard binding behaviour for functions implemented in Python.)|
|VAR_POSITIONAL|`'variadic positional'`|A tuple of positional arguments that aren’t bound to any other parameter. This corresponds to a *args parameter in a Python function definition.|
|KEYWORD_ONLY|`'keyword-only'`|Value must be supplied as a keyword argument. Keyword only parameters are those which appear after a * or *args entry in a Python function definition.|
|VAR_KEYWORD|`'variadic keyword'`|A dict of keyword arguments that aren’t bound to any other parameter. This corresponds to a **kwargs parameter in a Python function definition.|


**Example:** 

In [7]:
def foo(a, /, b, *c, d, e=10, **f):
    pass

sig = inspect.signature(foo)


*Count the number of parameters of each type*

In [8]:
Counter(param.kind.description for param in sig.parameters.values())


Counter({'keyword-only': 2,
         'positional-only': 1,
         'positional or keyword': 1,
         'variadic positional': 1,
         'variadic keyword': 1})

In [9]:
Counter(param.kind.name for param in sig.parameters.values())


Counter({'KEYWORD_ONLY': 2,
         'POSITIONAL_ONLY': 1,
         'POSITIONAL_OR_KEYWORD': 1,
         'VAR_POSITIONAL': 1,
         'VAR_KEYWORD': 1})

*List all parameters, their type and their default (if it exists):*

In [65]:
for param in sig.parameters.values():
    if param.default is param.empty:
        dft = ''
    else:
        dft = param.default
    print(f'{param.name}\t{param.kind.description:25s}{dft}')


a	positional-only          
b	positional or keyword    
c	variadic positional      
d	keyword-only             
e	keyword-only             10
f	variadic keyword         


*print all keyword-only arguments without default values:*

In [60]:
for param in sig.parameters.values():
    if (param.kind == param.KEYWORD_ONLY and
                       param.default is param.empty):
        print('Parameter:', param)


Parameter: d


*Calling signature:*
> `rule_method(test_object: Any, event: TriggerEvent, context)`

*Allowed signatures:*
> `func(arg1, event, context)`  *Both event and context as keyword args*<br>
> `func(arg1, arg2,   arg3  )`  *Final signature*<br>
> `func(arg1, arg2, **arg3  )`  *Immutable context*<br>
> `func(arg1, arg2          )`  *No context*<br>
> `func(arg1,       **arg3  )`  *No event*<br>
> `func(arg1                )`
> `func(arg1, event         )`  *event as keyword arg*<br>
> `func(arg1, context       )`  *context as keyword arg*<br>
> `func(arg1, context, event)`  *Reversed event and context keyword args*<br>

> `func(*args, **kwargs)`  *Generic*<br>

> `func(arg1, other_arg=`\<Any> ...`          )`  *Arguments with defaults*<br>
> `func(arg1, other_arg=`\<Any> ...`, **arg3  )`  *other_arg is taken from context (if present)*<br>



1. 'VAR_POSITIONAL' parameter present
   1. Number of parameters before 'VAR_POSITIONAL' parameter less than or equal to expected number.
   2. All parameter names after the 'VAR_POSITIONAL' parameter match.
   3. Required 'KEYWORD_ONLY' parameters present

2. 'VAR_KEYWORD' parameter present
   1. Number of positional parameters before 'VAR_KEYWORD' parameter less than or equal to expected number.
   2. 'context' is not the name of any parameter except 'VAR_KEYWORD'
   3. Required 'POSITIONAL_ONLY' parameters present

3. Number of parameters less than or equal to expected number.
   1. No unmatched parameter names present (order irrelevant)
      1. All parameter names present match.
      2. All required parameters present. 
   2. Unmatched parameter names present.  (order of matched parameter names irrelevant)
      1. All unmatched parameter names precede the matched parameter names.
      2. All unmatched parameter do not map by order to a matched parameter name. 
      3. Required 'KEYWORD_ONLY' parameters present.


4. Unmatched keyword parameter names present.
   1. All unmatched positional parameter names precede the matched parameter names.
   2. All unmatched parameters do not map by order to a matched parameter name. 
   3. Required 'KEYWORD_ONLY' parameters present.



5. Number of parameters greater than expected number.
6. All parameter names present match (but some non-required parameters missing, order still irrelevant)

- 'POSITIONAL_ONLY'
- 'POSITIONAL_OR_KEYWORD'
- 'VAR_POSITIONAL'
- 'KEYWORD_ONLY'
- 'VAR_KEYWORD'

In [11]:
def bar(a=1, b=2, c=3):
    print(a)
    print(b)
    print(c)

bar(b=5, c=6, a=4)

4
5
6


In [15]:
bar(5, c=7, b=6)

5
6
7


#### Testing supplied function signature
1. Check for matching argument names
2. check for VAR_KEYWORD

In [17]:
def parse_prescribed_dose(line, event, **context)->List[List[str]]:# pylint: disable=unused-argument
    '''Split "Prescribed dose [cGy]" into 2 lines:
        Prescribed dose
        Prescribed dose Unit
        '''
    parse_template = [
        ['Prescribed dose', '{dose}'],
        ['Prescribed dose Unit', '{unit}']
        ]
    match_results = event.test_value.groupdict()
    if match_results['dose'] == 'not defined':
        parsed_lines = [
            ['Prescribed dose', ''],
            ['Prescribed dose Unit', '']
            ]
    else:
        parsed_lines = [
            [string_item.format(**match_results) for string_item in line_tmpl]
            for line_tmpl in parse_template
            ]
    for line in parsed_lines:
        yield line


In [16]:
#%% Test Text
test_lines = '''
Prescribed dose [cGy]: 5000.0
Prescribed dose [cGy]: not defined

Plan Status: Unapproved
Plan Status: Treatment Approved Thursday, January 02, 2020 12:55:56 by gsal

% for dose(%): 100.0
% for dose (%): not defined
'''


In [18]:
inspect.isfunction(parse_prescribed_dose)

True

In [19]:
a = inspect.signature(parse_prescribed_dose)


In [22]:
str(a)

'(line, event, **context) -> List[List[str]]'

In [21]:
len(a.parameters)

3

In [34]:
b = [param.kind for param in a.parameters.values()]
b[0]

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [38]:
inspect._ParameterKind.__members__

mappingproxy({'POSITIONAL_ONLY': <_ParameterKind.POSITIONAL_ONLY: 0>,
              'POSITIONAL_OR_KEYWORD': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
              'VAR_POSITIONAL': <_ParameterKind.VAR_POSITIONAL: 2>,
              'KEYWORD_ONLY': <_ParameterKind.KEYWORD_ONLY: 3>,
              'VAR_KEYWORD': <_ParameterKind.VAR_KEYWORD: 4>})

In [31]:



Counter(param.kind.description for param in a.parameters.values())

Counter({'positional or keyword': 2, 'variadic keyword': 1})

In [ ]:
rule_sig = {
    (1, False): lambda func, item, event, context: func(item),
    (1, True): lambda func, item, event, context: func(item, **context),
    (2, False): lambda func, item, event, context: func(item, event),
    (2, True):
        lambda func, item, event, context: func(item, event, **context),
    (3, False):
        lambda func, item, event, context: func(item, event, context)
    }
process_sig = {
    (1, False): lambda func, item, context: func(item),
    (2, False): lambda func, item, context: func(item, context),
    (1, True): lambda func, item, context: func(item, **context)
    }


In [ ]:

arg_spec = inspect.getfullargspec(given_method)
# Determine arg_count
if not arg_spec.args:
    if arg_spec.varargs:
        arg_count = 1
    else:
        arg_count = 0
elif not arg_spec.defaults:
    arg_count = len(arg_spec.args)
else:
    arg_count = len(arg_spec.args) - len(arg_spec.defaults)
    if arg_count == 0:
        arg_count = 1

# Determine presence of keyword argument catcher
has_varkw = arg_spec.varkw is not None


In [ ]:
list(a.parameters.values())[1].name

'event'

In [ ]:
b = a.bind_partial('l')
b

<BoundArguments (line='l')>

In [ ]:
b.apply_defaults()
b

<BoundArguments (line='l', context={})>

In [ ]:
b.signature

<Signature (line, event, **context) -> List[List[str]]>

In [ ]:
def date_parse(line, event)->List[List[str]]:
    '''If Date,don't split beyond first :'''
    parsed_line = [event.test_value, line.split(':',maxsplit=1)[1]]
    return parsed_line


In [ ]:
def approved_status_parse(line, event)->List[List[str]]:
    '''If Treatment Approved, Split "Plan Status" into 3 lines:
        Plan Status
        Approved on
        Approved by
        '''
    idx1 = line.find(event.test_value)
    idx2 = idx1 + len(event.test_value)
    idx3 = line.find('by')
    idx4 = idx3 + 3
    parsed_lines = [
        ['Plan Status', line[idx1:idx2]],
        ['Approved on', line[idx2:idx3]],
        ['Approved by', line[idx4:]]
        ]
    for line in parsed_lines:
        yield line

In [ ]:
def make_float(item, context):
    item_c = item.replace('Line','')
    try:
        num = float(item_c.strip())
    except:
        return None
    output =  f'Line {num:5.2f}'
    context['num'] = num
    return output

In [ ]:

if sig_type == 'Process':
    sig_function = process_sig.get((arg_count, has_varkw))
else:
    sig_function = rule_sig.get((arg_count, has_varkw))
if not sig_function:
    raise ValueError('Invalid function type.')
use_function = partial(sig_function, given_method)
func_name = getattr(given_method, '__name__', None)
if not func_name:
    if isinstance(given_method, partial):
        func_name = getattr(given_method.func, '__name__',
                            'PartialFunction')
    else:
        func_name = 'PartialFunction'
use_function.__name__ = func_name
return use_function


In [ ]:
class TestRuleExceptions(unittest.TestCase):

    @unittest.skip('Currently rule_method(item, context) is allowed')
    def test_no_event_arg(self):
        test_func = lambda item, context: str(item) + repr(context)
        with self.assertRaises(ValueError):
            invalid_rule = Rule('T', pass_method=test_func)

    def test_no_arg(self):
        def test_func():
            return "T"
        with self.assertRaises(ValueError):
            invalid_rule = Rule('T', pass_method=test_func)

    def test_many_arg(self):
        test_func = lambda item1, item2, event, context: 'a'
        with self.assertRaises(ValueError):
            invalid_rule = Rule('T', pass_method=test_func)

    def test_bad_action(self):
        with self.assertRaises(ValueError):
            invalid_rule = Rule('T', pass_method='Not an Action')

class TestRuleActions(unittest.TestCase):
    def test_original_action(self):
        test_text = 'Test Text'
        test_rule = Rule('Text', pass_method='Original')
        output = iter(test_rule(test_text))
        result = next(output)
        self.assertEqual(result, test_text)

    def test_event_action(self):
        test_text = 'Test Text'
        sentinel = 'Text'
        test_rule = Rule(sentinel, pass_method='Event')
        output = iter(test_rule(test_text))
        result = next(output)
        self.assertEqual(result.test_value, sentinel)

    def test_none_action(self):
        test_text = 'Test Text'
        test_rule = Rule('Text', pass_method='None')
        output = iter(test_rule(test_text))
        result = next(output)
        self.assertIsNone(result)

    def test_blank_action(self):
        test_text = 'Test Text'
        test_rule = Rule('Text', pass_method='Blank')
        # Note this works because the Rule methods are not generator functions.
        result = test_rule.apply(test_text)
        self.assertEqual(result, '')

class TestRuleFail(unittest.TestCase):
    def test_fail_method(self):
        test_text = 'Test Line'
        test_rule = Rule('Text', pass_method='Blank',
                            fail_method='Original')
        # Note this works because the Rule methods are not generator functions.
        result = test_rule.apply(test_text, {})
        self.assertEqual(result, test_text)
        result2 = test_rule.apply('Test Text', {})
        self.assertEqual(result2, '')
